In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 256
batch_size = 16
max_iters = 5000
learning_rate = 1e-2
eval_iters = 250

cpu


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
with open("wizard_of_oz.txt", encoding='utf-8') as f:
    text = f.read()

In [ ]:
chars = sorted(set(text))
print(len(chars),chars)

In [ ]:
vocab_size = len(chars)

In [ ]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
data, len(text)

In [ ]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split:str):
  data = train_data if split=="train" else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i : i+block_size] for i in ix])
  y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
  x,y = x.to(device), y.to(device)
  return x,y

In [ ]:
x,y = get_batch('train')
# print(x)
# print(y)

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
              X,Y = get_batch(split)
              logits, loss = model(X,Y)
              losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape # batch, time, channels(vocab size)
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index) # get prediction
            logits = logits[:, -1, :] # get the last time step, (B,C)
            probs = F.softmax(logits, dim=-1) # apply softmax to get probs
            index_next = torch.multinomial(probs, num_samples=1) # sample from distribution (B,1)
            index = torch.cat((index, index_next), dim=1) # append sampled to get sequence (B, T+1)
        return index

In [ ]:
model = BigramLanguageModel(vocab_size)

In [ ]:
m = model.to(device)

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
context

In [ ]:
generate_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generate_chars)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_list=[]
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    loss_list.append(loss.item())

plt.plot(loss_list)

In [ ]:

    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    loss_list.append(loss.item())

plt.plot(loss_list)

In [ ]:
(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    loss_list.append(loss.item())

plt.plot(loss_list)

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

In [ ]:
context = torch.tensor([encode("The Wiz")], dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)